# Procesamiento del Lenguaje Natural
# Prueba de Evaluación Continua 2


**OBJETIVO 1**

Extracción de información en forma de palabras clave y entidades relevantes (Named Entities) con el objetivo de etiquetar los documentos de nuestro corpus.

**Instalamos las bibliotecas necesarias**

In [ ]:
!pip install nltk spacy scikit-learn pandas
!python -m spacy download en_core_web_sm


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/39/e1/08681583569f435347ced0535b27c073fcc9a927d9b4293c963092f2d01c/spacy-3.7.5-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for spacy-legacy<3.1.0,>=3.0.11 from https://files.pythonhosted.org/packages/c3/55/12e842c70ff8828e34e543a2c7176dac4da006ca6901c9e8b43efab8bc6b/spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata
  Obtaining dependency information for spacy-loggers<2.0.0,>=1.0.0 from https://files.pythonhosted.org/packages/33/78/d1a1a026ef3af911159398c939b1509d5c36fe524c7b644f34a5146c4e16/spacy_loggers-1.0.5-py3-none-any.whl.metadata
  Obtaining dependency information for murmurhash<1.1.0,>=0.28.0 from https://files.pythonhosted.org/packages/71/46/af01a20ec368bd9cb49a1d2df15e3eca113bbf6952cc1f2a47f1c6801a7f/murmurhash-1.0.10-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.1/12.8 MB 2.2 MB/s eta 0:00:06
      --------------------------------------- 0.2/12.8 MB 1.8 MB/s eta 0:00:07
      --------------------------------------- 0.3/12.8 MB 1.7 MB/s eta 0:00:08
     - -------------------------------------- 0.4/12.8 MB 1.9 MB/s eta 0:00:07
     - -------------------------------------- 0.6/12.8 MB 2.3 MB/s eta 0:00:06
     --- ------------------------------------ 1.1/12.8 MB 3.3 MB/s eta 0:00:04
     ---- ----------------------------------- 1.5/12.8 MB 3.9 MB/s eta 0:00:03
     ---- ----------------------------------- 1.5/12.8 MB 3.9 MB/s eta 0:00:03
     ---- ----------------------------------- 1.5/12.8 MB 3.9 MB/s eta 0:00:03
     -------- ------------------------------- 2.7/12.8 MB 4.5 MB/s eta 0:00:03
     ---------------- ----------------------- 5.2/12.8 MB 7.

In [ ]:
import pandas as pd
import spacy

# Cargar el archivo CSV
df = pd.read_csv('Reviews.csv')

# Mostrar las primeras filas del DataFrame
df.head()


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


**Cargamos el corpus escogido "Amazon Fine Food Reviews" tomado desde https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews/data**

In [ ]:
import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv('Reviews.csv')

# Mostrar las primeras filas del DataFrame
print(df.head())

# Descripción del corpus
corpus_size = len(df)
print(f"Tamaño del corpus: {corpus_size}")



   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1 

**Limpieza y preparación de los datos**

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.tokens import Token

# Cargar el modelo de spaCy
nlp = spacy.load('en_core_web_sm')

# Función de preprocesamiento
def preprocess_text(text):
    # Normalización: convertir a minúsculas
    text = text.lower()

    # Tokenización y análisis con spaCy
    doc = nlp(text)

    # Eliminación de stop words y puntuación
    tokens = [token for token in doc if token.text not in STOP_WORDS and not token.is_punct]

    # Lematización
    lemmatized_tokens = [token.lemma_ for token in tokens]

    return ' '.join(lemmatized_tokens)

# Aplicar la función de preprocesamiento a la columna 'Text'
df_sample['Cleaned_Text'] = df_sample['Text'].apply(preprocess_text)

# Mostrar las primeras filas del DataFrame con el texto preprocesado
print(df_sample[['Text', 'Cleaned_Text']].head())


                                                     Text  \
165256  Having tried a couple of other brands of glute...   
231465  My cat loves these treats. If ever I can't fin...   
427827  A little less than I expected.  It tends to ha...   
433954  First there was Frosted Mini-Wheats, in origin...   
70260   and I want to congratulate the graphic artist ...   

                                             Cleaned_Text  
165256  having try couple brand gluten free sandwich c...  
231465  cat love treat find house pop bolt hide come t...  
427827  little expect   tend muddy taste expect say fa...  
433954  frosted mini wheat original size frosted mini ...  
70260   want congratulate graphic artist put entire pr...  


**Extracción de palabras clave (KPE) y PoS tagging con librerías pre-entrenadas**

In [ ]:
# Función para extraer palabras clave y entidades nombradas del texto preprocesado
def extract_keywords_entities(text):
    doc = nlp(text)
    keywords = [token.text for token in doc if token.pos_ in ['NOUN', 'ADJ']]
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return keywords, entities

# Aplicar la función a la columna 'Cleaned_Text'
df_sample['Keywords_Entities'] = df_sample['Cleaned_Text'].apply(extract_keywords_entities)

# Mostrar los resultados
print(df_sample[['Cleaned_Text', 'Keywords_Entities']].head())



                                             Cleaned_Text  \
165256  having try couple brand gluten free sandwich c...   
231465  cat love treat find house pop bolt hide come t...   
427827  little expect   tend muddy taste expect say fa...   
433954  frosted mini wheat original size frosted mini ...   
70260   want congratulate graphic artist put entire pr...   

                                        Keywords_Entities  
165256  ([couple, brand, free, cookie, good, bunch, cr...  
231465  ([treat, house, pop, bolt, hide, crunchy, trea...  
427827    ([little, muddy, taste, favorite, company], [])  
433954  ([mini, wheat, original, size, mini, wheat, bi...  
70260   ([graphic, artist, entire, product, small, box...  


**Reconocimiento de entidades relevantes (NER) por medio de heurísticas, regexp, etiquetas PoS, lexicones, etc. En este caso nos enfocamos en los atributos de los productos**

In [ ]:
import re

# Lista de posibles atributos de productos
attributes = ['taste', 'price', 'quality', 'packaging', 'flavor', 'texture', 'smell', 'appearance', 'size', 'value']

# Función para encontrar posibles atributos de productos utilizando heurísticas y regexp
def find_product_attributes(text):
    doc = nlp(text)
    attributes_found = []

    # Heurística: Buscar patrones comunes en los atributos de productos
    pattern = re.compile(r'\b(?:' + '|'.join(attributes) + r')\b', re.IGNORECASE)

    for sent in doc.sents:
        if pattern.search(sent.text):
            # Incluir tokens etiquetados como sustantivos y adjetivos relevantes
            for token in sent:
                if token.lemma_ in attributes:
                    attributes_found.append(token.text.strip())
    return list(set(attributes_found))  # Eliminar duplicados

# Aplicar la función a la columna 'Cleaned_Text'
df_sample['Product_Attributes'] = df_sample['Cleaned_Text'].apply(find_product_attributes)

# Mostrar los resultados
print(df_sample[['Cleaned_Text', 'Product_Attributes']].head())


                                             Cleaned_Text  \
165256  having try couple brand gluten free sandwich c...   
231465  cat love treat find house pop bolt hide come t...   
427827  little expect   tend muddy taste expect say fa...   
433954  frosted mini wheat original size frosted mini ...   
70260   want congratulate graphic artist put entire pr...   

           Product_Attributes  
165256       [taste, texture]  
231465               [flavor]  
427827                [taste]  
433954  [flavor, size, taste]  
70260                 [taste]  


In [ ]:
from transformers import pipeline

# Cargar pipeline de NER de HuggingFace
ner_pipeline = pipeline("ner", model="dslim/bert-base-NER")

# Función para aplicar NER
def apply_ner(text):
    ner_results = ner_pipeline(text)
    return [(result['word'], result['entity']) for result in ner_results]

# Aplicar la función NER a la columna 'Cleaned_Text'
df_sample['NER_Entities'] = df_sample['Cleaned_Text'].apply(apply_ner)

# Mostrar los resultados
print(df_sample[['Cleaned_Text', 'NER_Entities']].head())



Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


                                             Cleaned_Text NER_Entities
165256  having try couple brand gluten free sandwich c...           []
231465  cat love treat find house pop bolt hide come t...           []
427827  little expect   tend muddy taste expect say fa...           []
433954  frosted mini wheat original size frosted mini ...           []
70260   want congratulate graphic artist put entire pr...           []


**OBJETIVO 2**

**Realizar fine-tuning para clasificación sobre un corpus y modelo LLM pre-entrenado de la plataforma HuggingFace**

instalamos dependencias

In [ ]:
!pip install requests==2.31.0
!pip install transformers==4.40.2 datasets accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (41

**Importamos librerías necesarias**

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer


**Cargamos y dividimos el Dataset mteb/mtop_domain**

In [ ]:
dataset = load_dataset("mteb/mtop_domain")

train_dataset = dataset['train'].shuffle(seed=42).select(range(5000))  # Selecciona una muestra de 5000 para entrenamiento
test_dataset = dataset['validation'].shuffle(seed=42).select(range(1000))  # Selecciona una muestra de 1000 para validación


/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for mteb/mtop_domain contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mteb/mtop_domain
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

**Cargamos el Tokenizer y el modelo precargado**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=11)  # Ajustar el número de etiquetas según el dataset


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Procesamos los datos**

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

**configuramos los argumentos de entrenamiento**

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)


**Entrenamos y evaluamos el modelo**

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

results = trainer.evaluate()
print(results)


Epoch,Training Loss,Validation Loss
1,No log,0.110384
2,0.516500,0.069268
3,0.516500,0.063906


{'eval_loss': 0.06390639394521713, 'eval_runtime': 16.9562, 'eval_samples_per_second': 58.975, 'eval_steps_per_second': 3.715, 'epoch': 3.0}
